In [10]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['trip duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['trip duration'] = df['trip duration'].apply(lambda x: x.total_seconds() / 60)

    dfClean = df[(df['trip duration'] >= 1) & (df['trip duration'] < 60)]

    categorical = ['PULocationID','DOLocationID']

    dfClean[categorical] = dfClean[categorical].astype(str)
    return df

In [4]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_test = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')

/tmp/ipykernel_14675/674562825.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfClean[categorical] = dfClean[categorical].astype(str)
/tmp/ipykernel_14675/674562825.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfClean[categorical] = dfClean[categorical].astype(str)


In [5]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_test['PU_DO'] = df_test['PULocationID'].astype(str) + '_' + df_test['DOLocationID'].astype(str)

In [7]:
dv = DictVectorizer()
categorical = ['PU_DO']#PULocationID','DOLocationID']
numerical = ['trip_distance']

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'trip duration'
y_train = df_train[target].values
y_val = df_test[target].values

In [9]:
lr = Ridge(alpha = 0.01)
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

32.93439925406707

In [ ]:
sns.distplot(y_pred, label='predict')
sns.distplot(y_train, label= 'actual')

In [ ]:
with open('model/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr))